In [5]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import Markdown as md
import pandas as pd

InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

time_zone = 'Asia/Jakarta'

In [6]:
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter


class CacheLimiterSession(CacheMixin, LimiterMixin, Session):
    pass


request_limit = 2
request_interval = 5 * Duration.SECOND
db_path = ("yfinance.cache")


def generate_session() -> CacheLimiterSession:
    return CacheLimiterSession(
        limiter=Limiter(RequestRate(limit=request_limit, interval=request_interval)),
        bucket_class=MemoryQueueBucket,
        backend=SQLiteCache(db_path=db_path)
    )


session = generate_session()

In [7]:
import data_source
from datetime import datetime, timedelta
import yfinance as yf

In [8]:
return_column_name = 'return'

In [9]:
def generate_returns_of_market(start_time: datetime, end_time: datetime, session=generate_session()) -> pd.DataFrame:
    market_ticker = data_source.ticker_market
    market = yf.Ticker(ticker=market_ticker, session=session)

    result = market.history(start=start_time, end=end_time)
    returns_of_market = result.copy()['Close'].pct_change()[1:].to_frame()

    first_column_name = returns_of_market.columns[0]
    returns_of_market.rename(columns={first_column_name: return_column_name}, inplace=True)

    return returns_of_market

In [10]:
def generate_stock_prices(start_time: datetime, end_time: datetime, tickers: pd.DataFrame, \
                          session=generate_session()) -> pd.DataFrame:
    classified_tickers_list = tickers['ticker'].to_list()

    tickers_prices = yf.download(tickers=classified_tickers_list, start=start_time, end=end_time, session=session)
    tickers_close_prices = tickers_prices.copy()['Close'].dropna(axis=1)

    return tickers_close_prices

In [11]:
def calculate_tickers_return(tickers_stock_prices: pd.DataFrame) -> pd.DataFrame:
    tickers_return = tickers_stock_prices.copy().pct_change()[1:]
    return tickers_return

In [12]:
date_column_name = 'date'
ticker_column_name = 'ticker'
return_column_name = 'return'
observation_param_name_for_return = 'param1_r'

In [13]:
def init_observation_df(input_df: pd.DataFrame) -> pd.DataFrame:
  observation_list = []
  for period, row in input_df.iterrows():
    period = period.tz_localize(time_zone)
    for ticker, daily_return in row.items():
      observation_list.append({
          date_column_name: period,
          ticker_column_name: ticker,
          return_column_name: daily_return
      })
  observation_raw_df = pd.DataFrame(observation_list)

  # raw pivot
  observation_df = observation_raw_df.copy().pivot_table(index=date_column_name, columns=ticker_column_name)

  # fixed_pivot
  observation_df.rename(columns={return_column_name: observation_param_name_for_return}, inplace=True)
  observation_df = observation_df.swaplevel(i=0, j=1, axis=1)

  return observation_df

In [14]:
observation_param_name_for_u = 'param2_u'

In [15]:
def calculate_u(observation_df: pd.DataFrame, returns_of_market: pd.DataFrame) -> pd.DataFrame:
  for column, _ in observation_df.items():
    _ticker, _ = column
    observation_df[_ticker, observation_param_name_for_u] = observation_df.copy()[_ticker, observation_param_name_for_return] \
      - returns_of_market[return_column_name]
  observation_df = observation_df.copy().reindex(sorted(observation_df.copy().columns), axis=1)
  return observation_df

In [16]:
observation_param_name_for_cu = 'param3_cu'

In [17]:
def calculate_cu(observation_df: pd.DataFrame) -> pd.DataFrame:
  for column, _ in observation_df.items():
    _ticker, _ = column
    observation_df[_ticker, observation_param_name_for_cu] = 0
    cu = 0
    for i, _ in observation_df[_ticker, observation_param_name_for_u].items():
      cu += observation_df.copy().at[i, (_ticker, observation_param_name_for_u)]
      observation_df.at[i, (_ticker, observation_param_name_for_cu)] = cu
  observation_df = observation_df.copy().reindex(sorted(observation_df.columns), axis=1)
  return observation_df

In [18]:
factor = 30
last_cu_column_name = 'last_cu'

In [ ]:
def sort_by_last_formation_day(observation_df: pd.DataFrame) -> pd.DataFrame:
    last_formation_day = observation_df.copy().iloc[-1]
    last_formation_day_dict = last_formation_day.to_dict()
    last_formation_day_list = [{ticker_column_name: key[0], last_cu_column_name: value} for key, value in last_formation_day_dict.items() if key[1] == 'param3_cu']
    last_formation_day_df = pd.DataFrame(last_formation_day_list).sort_values(by=last_cu_column_name, ascending=False)
    tickers_sorted = last_formation_day_df[ticker_column_name].to_list()
    return observation_df.copy()[tickers_sorted]

In [19]:
def select_winner_and_loser_tickers(observation_df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    last_formation_day = observation_df.iloc[-1]
    last_formation_day_dict = last_formation_day.to_dict()
    last_formation_day_list = [{ticker_column_name: key[0], last_cu_column_name: value} for key, value in last_formation_day_dict.items() if key[1] == 'param3_cu']
    last_formation_day_df = pd.DataFrame(last_formation_day_list).copy().sort_values(by=last_cu_column_name, ascending=False)
    ticker_winner_list = last_formation_day_df.iloc[:factor][ticker_column_name].to_list()
    ticker_loser_list = last_formation_day_df.iloc[-factor:][ticker_column_name].to_list()
    return observation_df[ticker_winner_list], observation_df[ticker_loser_list]